<a href="https://colab.research.google.com/github/vir-k01/StyleGAN2-ADA-MicrostructureGeneration/blob/main/GAN_MicrostructureGeneration.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Microstructure Generation using a trained StyleGAN2

Code accompanying the manuscript: "Quantification of similarity and physical awareness of microstructures generated via Generative Models" by Sanket Thakre, Vir Karan, Anand Krishna Kanjarla.

-Notebook authored by Vir Karan

## The Basics

This notebook contains the code needed to generate new microstructural data using a trained StyleGAN2-ADA model in the Google Colab environment. The notebook needed to train the GAN is present in the same repository as this (https://github.com/vir-k01/StyleGAN2-ADA-MicrostructureGeneration/blob/main/StyleGAN2_ADA_Training.ipynb). This notebook clones the official repository by Nvidia (https://github.com/NVlabs/stylegan2-ada) and then calls the scripts from that repo. Relevant changes can be made to run this locally or on a workstation. 

**StyleGAN2-ADA only work with Tensorflow 1. Run the next cell before anything else to make sure we’re using TF1 and not TF2.**

In [ ]:
%tensorflow_version 1.x

Make sure that a GPU runtime has been selected. GANs require **a lot** of compute to train, so make sure you've got a GPU with atleast 12GB RAM here. Even though we're not training a GAN here, having a GPU will be preferable since we'll be using the `dnnlib` functions next. In the case of Colab, the default K80 GPU should suffice for smaller datasets, but it is preferable to use a T4 or P100 or better GPU. We'll first clone the official StyleGAN2-ADA repo and load all the dependecies.

In [ ]:
!git clone https://github.com/NVlabs/stylegan2-ada.git
%cd stylegan2-ada
!nvcc test_nvcc.cu -o test_nvcc -run

import tensorflow as tf
print('Tensorflow version: {}'.format(tf.__version__) )
!nvidia-smi -L
print('GPU Identified at: {}'.format(tf.test.gpu_device_name()))

To allow for easy  loading/saving of files, it is preferable to mount Google Drive as well. 

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# Import dependencies from the repo and otherwise
import argparse
import numpy as np
import PIL
import PIL.Image
import dnnlib
import dnnlib.tflib as tflib
import re
import sys
from io import BytesIO
import IPython.display
from math import ceil
import imageio
import os
import pickle
import matplotlib.image as mpimg
from skimage import filters

Next, we'll load the trained model from Drive. Once again reiterating, **make sure a GPU is available** before going further. The `.pkl` file used can be found in our repo as well (https://drive.google.com/drive/folders/1Oz2NlycaIfK0-mebNgz5HTTW4_UuVikQ?usp=sharing).

In [ ]:
%cd /content/stylegan2-ada/

dnnlib.tflib.init_tf() 
network_pkl = '/content/drive/MyDrive/colab-sg2-ada/stylegan2-ada/results/00045-dam_data256-11gb-gpu-bg-resumecustom/network-snapshot-000020.pkl'
 
print('Loading networks from "%s"...' % network_pkl)
with dnnlib.util.open_url(network_pkl) as fp:
    _G, _D, Gs = pickle.load(fp)
noise_vars = [var for name, var in Gs.components.synthesis.vars.items() if name.startswith('noise')]

## Helper Functions

These custom functions allow us to generate and post-process the microstructures. 

In [ ]:
def rgb2gray(rgb):
    return np.dot(rgb[...,:3], [0.2989, 0.5870, 0.1140])  

def store_array(im, filename):
  fil = open(str(filename) + '.txt', 'w')
  row, column = im.shape
  for y in range(column):
      for x in range(row):
          pixel = im[y, x]
          fil.write(str(x+1) + " " + str(y+1) + " " + str(pixel) + '\n')
  fil.close()

def load_latent(path):
  latent = np.load(path, mmap_mode = 'r')
  return latent['dlatents']

def process_output(path, image = True):
  img = Image.open(path)
  return rgb2gray(np.asarray(img))/255

def threshold(arr, show = True, show_pf = 0):
  thres = filters.threshold_otsu(arr)
  bin = []
  for i in range(np.shape(arr)[0]):
    for j in range(np.shape(arr)[1]):
      if arr[i,j]> thres:
        bin.append(1)
      else:
        bin.append(0)
  bin = np.reshape(np.asarray(bin), (256, 256))
  pf = np.sum(bin)/(256*256)
  if show_pf:
    print('pf: ' + str(pf))
  if show:
    plt.imshow(bin, cmap = 'gray')
  return bin, pf

def generate_from_latent(latent, psi =0.7, show = True, save =False):
  imgs = Gs.components.synthesis.run(latent, output_transform=dict(func=tflib.convert_images_to_uint8, nchw_to_nhwc=True))
  im, pf = threshold(rgb2gray(imgs[0]), show = show)
  if show:
    plt.imshow(np.asarray(im), cmap = 'gray')
  if save:
    plt.imsave('output_'+ str(psi), np.asarray(im))
    imageio.imwrite('output_'+ str(psi), np.asarray(im))
  return np.asarray(im)

from mpl_toolkits.axes_grid1 import ImageGrid
def make_grid(img, rows, cols):
  fig = plt.figure(figsize=(40., 40.))
  grid = ImageGrid(fig, 111,  # similar to subplot(111)
                  nrows_ncols=(rows, cols),  # creates 2x2 grid of axes
                  axes_pad=0.1,  # pad between axes in inch.
                  )

  for ax, im in zip(grid, img):
      # Iterating over the grid returns the Axes.
      ax.imshow(im, cmap='gray')

def generate_images_direction(latent, direction, distance, scale, show =1, compare =0):
  latent1 = latent + direction*scale*distance 
  imgs = Gs.components.synthesis.run(latent1, output_transform=dict(func=tflib.convert_images_to_uint8, nchw_to_nhwc=True))
  img, pf = threshold(rgb2gray(imgs[0]), show = show)
  if compare:
    img0 = Gs.components.synthesis.run(latent, output_transform=dict(func=tflib.convert_images_to_uint8, nchw_to_nhwc=True))
    img0, pf1 = threshold(rgb2gray(img0[0]), show = show)
    compare_image(img0, img, '')
  return img

def generate_n_images(latent, n=5, show = 1, noise=1):
  out = []
  for i in range(n):
    latent1 = latent + 0.01*np.random.randn(1, 14, 512)*noise
    imgs = Gs.components.synthesis.run(latent1, output_transform=dict(func=tflib.convert_images_to_uint8, nchw_to_nhwc=True))
    img, pf = threshold(rgb2gray(imgs[0]), show = show)
    out.append(img)
  return out

## Projecting to the latent space

Before going further, in order to have control over the microstructures that are generated, we'll have to project our target microstructures into the trained GAN's latent space and get a latent code (section 2.2 of the manuscript). This process has to be done only once. You can skip this if you've got the latent code for the target microstructure you want. Regardless, a random (1, 14, 512) vector can also be used to generate microstructures.

Just define lists with the class_id and the indices of the microstructures whose latent representations you need. 

In [ ]:
class_id = [1, 2, 3, 4, 5, 6, 1, 2, 3, 4, 5, 6, 1, 2, 3, 4, 5, 6, 1, 2, 3, 4, 5, 6, 1, 2, 3, 4, 5, 6]
index = [1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 3, 3, 3, 3, 3, 3, 4, 4, 4, 4, 4, 4, 5, 5, 5, 5, 5, 5]

We'll now use the `projector.py` script from the repo to perform the projection. You can check the arguements for this script by running the next cell.

In [ ]:
!python projector.py -h

Load the trained model with the "network" tag, loop over all the classes and project one microstructure from each class using the "target tag" to get a "representative" latent vector for that class and save in the "outdir" tag.

Note: each microstructure would take around 2-5 min on a T4 GPU to project into the latent space.

In [ ]:
for i, j in zip(index, class_id):
  !python projector.py --outdir='/content/drive/MyDrive/GAN/Damage prediction/Out/latents/'{i}'_class_'{j}'/' --target='/content/drive/MyDrive/GAN/damage_data/'{50*(j-1)+(i-1)}'.jpeg' --network='/content/drive/MyDrive/colab-sg2-ada/stylegan2-ada/results/00045-dam_data256-11gb-gpu-bg-resumecustom/network-snapshot-000020.pkl'
  print('Finished: '+ str(i)+' '+ str(j))

We've put up the latent vectors corresponding to the microstructures of our study on the repo as well (https://drive.google.com/drive/folders/1Oz2NlycaIfK0-mebNgz5HTTW4_UuVikQ?usp=sharing).

## An example showing how to generate microstructures using the latent vector and the trained model



If you don't have a latent code already, just sample a (1, 14, 512) dimension vector from a Gaussian and use that as the latent vector. Here, we'll use the latent vector for Class 5. 

In [ ]:
lat = np.load('/content/drive/MyDrive/GAN/Vir files/Damage prediction/Out/latents/14_class_5/dlatents.npz')['dlatents']
c7_50 = generate_n_images(lat)
make_grid(c7_50, 1, 5)

Once generated, you can export the microstructures to a `.txt` file for further processing and analysis.

In [ ]:
for i in range(5):
  store_array(c7_50[i], '/content/drive/MyDrive/GAN/Vir files/generated/class_7_50_'+str(i))

You can uncomment the next cell and generate a desired microstructure using just one line of code!

In [ ]:
#latent_loc = ''
#save_loc = ''


#store_array(generate_n_images(np.load(latent_loc)['dlatents'])[0], save_loc)